In [ ]:
import os
import json
import re

import numpy as np
import pandas as pd
import torch
import scanpy as sc
from torchvision.datasets.folder import default_loader

In [ ]:
from stainaug import Augmentor

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
from violet.utils.dataloaders import listfiles

In [ ]:
fps = list(listfiles('../data/st/pdac_ffpe_07292021/train/HT270P1_S1H1Fs5U1/', regex='.jpeg$'))
len(fps)

In [ ]:
fps[:10]

In [ ]:
img = default_loader(fps[70])
img

In [ ]:
augmentor = Augmentor()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for i in range(10):
    plt.imshow(augmentor.augment_HE(np.asarray(img)))
    plt.show()

In [ ]:
from violet.utils.dataloaders import dino_he_transform_with_aug

In [ ]:
t = dino_he_transform_with_aug()

In [ ]:
from einops import rearrange

In [ ]:
img = default_loader(fps[-1])
img

In [ ]:
img = default_loader(fps[-10])
img

In [ ]:
for x in range(10):
    plt.imshow(rearrange(t(img).numpy(), 'c h w -> h w c'))
    plt.show()

In [ ]:
img = default_loader(fps[40])
img

In [ ]:
for x in range(10):
    tim = t(img).numpy()
    print(np.min(tim), np.max(tim))
    plt.imshow(rearrange(tim, 'c h w -> h w c'))
    plt.show()

In [ ]:
import numpy as np
from violet.utils.dino_utils import HE_color_transform

In [ ]:
# compute new means and stds
tcia_fps = sorted(listfiles('/data/tcia/PDA_preprocessed_small_raw/', regex=r'.jpeg$'))
inhouse_fps = sorted(listfiles('../data/st/pdac_ffpe_07292021/train/', regex=r'.jpeg$'))
tcia_v_fps = sorted(listfiles('/data/tcia/PDA_preprocessed_small/', regex=r'.jpeg$'))
inhouse_v_fps = sorted(listfiles('../data/st/pdac_ffpe_normalized/train/', regex=r'.jpeg$'))

In [ ]:
from PIL import Image
from violet.utils.dino_utils import HE_color_transform

In [ ]:
import PIL

In [ ]:
fp = tcia_fps[10]
img = Image.open(fp)
img

In [ ]:
'PIL' in str(type(img))

In [ ]:
t = HE_color_transform()

In [ ]:
t(img)

In [ ]:
import torchvision

In [ ]:
to_tensor = torchvision.transforms.ToTensor()

In [ ]:
img = to_tensor(img)

In [ ]:
isinstance(img, torch.Tensor)

In [ ]:
img.shape

In [ ]:
aug = t(img)
aug.shape

In [ ]:
to_pil = torchvision.transforms.ToPILImage()

In [ ]:
to_pil(aug)

In [ ]:
aug

In [ ]:
sum_means, sum_stds = np.asarray([0., 0., 0.]), np.asarray([0., 0., 0.])
n = 1000
ct = HE_color_transform()
for fp in np.random.choice(tcia_fps, size=n):
    img = default_loader(fp)
    img = r(img)
    img = hec(img)
#     img = img.numpy()
    sum_means += np.mean(rearrange(np.asarray(img), 'c h w -> (h w) c'), axis=0)
    sum_stds += np.std(rearrange(np.asarray(img), 'c h w -> (h w) c'), axis=0)
sum_means / n, sum_stds / n

In [ ]:
sum_means, sum_stds = np.asarray([0., 0., 0.]), np.asarray([0., 0., 0.])
n = 1000
ct = HE_color_transform()
for fp in np.random.choice(inhouse_fps, size=n):
    img = default_loader(fp)
    img = r(img)
    img = hec(img)
#     img = img.numpy()
    sum_means += np.mean(rearrange(np.asarray(img), 'c h w -> (h w) c'), axis=0)
    sum_stds += np.std(rearrange(np.asarray(img), 'c h w -> (h w) c'), axis=0)
sum_means / n, sum_stds / n

In [ ]:
# sum_means, sum_stds = np.asarray([0., 0., 0.]), np.asarray([0., 0., 0.])
# n = 10000
# ct = HE_color_transform()
# for fp in np.random.choice(tcia_v_fps, size=n):
#     img = torch.tensor(np.asarray(default_loader(fp)))
# #     img = ct(img).numpy()
#     img = img.numpy()
#     sum_means += np.mean(rearrange(np.asarray(img), 'h w c -> (h w) c'), axis=0)
#     sum_stds += np.std(rearrange(np.asarray(img), 'h w c -> (h w) c'), axis=0)
# sum_means / n / 255., sum_stds / n / 255.

In [ ]:
# sum_means, sum_stds = np.asarray([0., 0., 0.]), np.asarray([0., 0., 0.])
# n = 10000
# ct = HE_color_transform()
# for fp in np.random.choice(inhouse_v_fps, size=n):
#     img = torch.tensor(np.asarray(default_loader(fp)))
# #     img = ct(img).numpy()
#     img = img.numpy()
#     sum_means += np.mean(rearrange(np.asarray(img), 'h w c -> (h w) c'), axis=0)
#     sum_stds += np.std(rearrange(np.asarray(img), 'h w c -> (h w) c'), axis=0)
# sum_means / n / 255., sum_stds / n / 255.

In [ ]:
from violet.utils.dino_utils import DataAugmentationDINOMultichannel, HE_color_transform

In [ ]:
from torchvision import datasets, transforms
inhouse = transforms.Normalize(
    (0.26181996, 0.59005849, 0.32784506), (0.16256156, 0.16977168, 0.15752241)
)
tcia = transforms.Normalize(
    (0.24834598, 0.47750344, 0.35384849), (0.12277239, 0.14210545, 0.12420313)
)
r = transforms.Compose([
    transforms.Resize((224, 224), interpolation=3),
    transforms.ToTensor()
])
hec = HE_color_transform()

In [ ]:
n = 1
for fp in np.random.choice(inhouse_fps, size=n):
    for i in range(10):
        img = default_loader(fp)
        img = r(img)
        img = hec(img)
        plt.imshow(rescale_intensity(rearrange(img.numpy(), 'c h w -> h w c'), out_range=(0., 1.)))
        plt.show()

In [ ]:
from skimage.exposure import rescale_intensity

In [ ]:
n = 1
ct = HE_color_transform()
for fp in np.random.choice(inhouse_fps, size=n):
    img = default_loader(fp)
    img = r(img)
    print(img.shape)
    
    print('raw')
    plt.imshow(rearrange(img.numpy(), 'c h w -> h w c'))
    plt.show()
    
    print('aug')
    aug = hec(img)
    plt.imshow(rescale_intensity(rearrange(aug.numpy(), 'c h w -> h w c'), out_range=(0., 1.)))
    plt.show()
    
    print(torch.max(aug))
    
    print('aug inhouse norm')
#     img_inhouse = hec(img)
    img_inhouse = inhouse(aug)
    plt.imshow(rescale_intensity(rearrange(img_inhouse.numpy(), 'c h w -> h w c'), out_range=(0., 1.)))
    plt.show()    
    
    print('aug tcia norm')
    img_tcia = tcia(aug)
    plt.imshow(rescale_intensity(rearrange(img_tcia.numpy(), 'c h w -> h w c'), out_range=(0., 1.)))
    plt.show() 

In [ ]:
n = 1
ct = HE_color_transform()
for fp in np.random.choice(tcia_fps, size=n):
    img = default_loader(fp)
    img = r(img)
    print(img.shape)
    
    print('raw')
    plt.imshow(rearrange(img.numpy(), 'c h w -> h w c'))
    plt.show()
    
    print('aug')
    aug = hec(img)
    plt.imshow(rescale_intensity(rearrange(aug.numpy(), 'c h w -> h w c'), out_range=(0., 1.)))
    plt.show()
    
    print(torch.max(aug))
    
    print('aug inhouse norm')
#     img_inhouse = hec(img)
    img_inhouse = inhouse(aug)
    plt.imshow(rescale_intensity(rearrange(img_inhouse.numpy(), 'c h w -> h w c'), out_range=(0., 1.)))
    plt.show()    
    
    print('aug tcia norm')
    img_tcia = tcia(aug)
    plt.imshow(rescale_intensity(rearrange(img_tcia.numpy(), 'c h w -> h w c'), out_range=(0., 1.)))
    plt.show() 

In [ ]:
he = dino_he_transform_with_aug()

In [ ]:
n = 1
for fp in np.random.choice(inhouse_fps, size=n):
    img = default_loader(fp)
    plt.imshow(np.asarray(img))
    plt.show()
    for i in range(10):
        img = default_loader(fp)
        img = he(img)
        plt.imshow(rescale_intensity(rearrange(img.numpy(), 'c h w -> h w c'), out_range=(0., 1.)))
        plt.show()

In [ ]:
n = 1
for fp in np.random.choice(inhouse_fps, size=n):
    img = default_loader(fp)
    plt.imshow(np.asarray(img))
    plt.show()
    for i in range(10):
        img = default_loader(fp)
        img = he(img)
        plt.imshow(rescale_intensity(rearrange(img.numpy(), 'c h w -> h w c'), out_range=(0., 1.)))
        plt.show()

In [ ]:
n = 1
for fp in np.random.choice(tcia_fps, size=n):
    img = default_loader(fp)
    plt.imshow(np.asarray(img))
    plt.show()
    for i in range(10):
        img = default_loader(fp)
        img = he(img)
        plt.imshow(rescale_intensity(rearrange(img.numpy(), 'c h w -> h w c'), out_range=(0., 1.)))
        plt.show()

In [ ]:
n = 1
for fp in np.random.choice(tcia_fps, size=n):
    img = default_loader(fp)
    plt.imshow(np.asarray(img))
    plt.show()
    for i in range(10):
        img = default_loader(fp)
        img = he(img)
        plt.imshow(rescale_intensity(rearrange(img.numpy(), 'c h w -> h w c'), out_range=(0., 1.)))
        plt.show()